<a href="https://colab.research.google.com/github/Bhuvan-66/NM_churn/blob/main/sms_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Step 1: Import libraries
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Download stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# ✅ Step 2: Load dataset (correct columns)
df = pd.read_csv('/content/spam.csv', encoding='latin1')

# Keep only first two useful columns and rename them
df = df[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'message'})
df = df.dropna()  # remove any empty rows

print("✅ Dataset loaded successfully!")
print(df.head())

# ✅ Step 3: Clean the text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)   # remove URLs
    text = re.sub(r'[^a-z\s]', ' ', text)        # keep only alphabets
    text = re.sub(r'\s+', ' ', text).strip()     # remove extra spaces
    return text

df['clean_message'] = df['message'].apply(clean_text)

# ✅ Step 4: Encode labels
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

# ✅ Step 5: Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_message'], df['label_num'], test_size=0.2, random_state=42
)

# ✅ Step 6: TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=3000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# ✅ Step 7: Train the model
model = MultinomialNB()
model.fit(X_train_vec, y_train)
print("\n✅ Model training completed!")

# ✅ Step 8: Evaluate
y_pred = model.predict(X_test_vec)
print("\n📊 Model Evaluation:")
print("Accuracy:", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("\n", classification_report(y_test, y_pred))

# ✅ Step 9: Interactive input
print("\n--- SMS Spam Checker ---")
while True:
    msg = input("Enter an SMS (or 'exit' to quit): ")
    if msg.lower() == 'exit':
        print("Exiting... 👋")
        break
    clean_msg = clean_text(msg)
    vec = vectorizer.transform([clean_msg])
    prob = model.predict_proba(vec)[0][1]  # spam probability
    print("Spam Probability:", round(prob*100, 2), "%")
    pred = model.predict(vec)[0]
    print("Result:", "🚨 Spam 🚨" if pred == 1 else "✅ Not Spam (Ham)")
    print("-" * 40)

import joblib

joblib.dump(model, 'spam_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

from google.colab import files
files.download('spam_model.pkl')
files.download('vectorizer.pkl')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Dataset loaded successfully!
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

✅ Model training completed!

📊 Model Evaluation:
Accuracy: 97.49 %

               precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.81      0.90       150

    accuracy                           0.97      1115
   macro avg       0.99      0.91      0.94      1115
weighted avg       0.98      0.97      0.97      1115


--- SMS Spam Checker ---
Enter an SMS (or 'exit' to quit): hi how are you
Spam Probability: 3.69 %
Result: ✅ Not Spam (Ham)
----------------------------------------


In [10]:
# STEP 1: Import libraries
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Download stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# STEP 2: Load dataset
df = pd.read_csv('/content/spam.csv', encoding='latin1')
df = df[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'message'})
df = df.dropna()

# STEP 3: Clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_message'] = df['message'].apply(clean_text)
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

# STEP 4: Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_message'], df['label_num'], test_size=0.2, random_state=42, stratify=df['label_num']
)

# STEP 5: Better vectorizer (include bigrams + more features)
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# STEP 6: Stronger model (Logistic Regression)
model = LogisticRegression(max_iter=2000, class_weight='balanced')  # handles imbalance
model.fit(X_train_vec, y_train)

# STEP 7: Evaluate
y_pred = model.predict(X_test_vec)
print("✅ Model Training Complete!")
print("\nAccuracy:", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# STEP 8: Test interactive input
print("\n--- SMS Spam Checker ---")
while True:
    msg = input("Enter an SMS (or 'exit' to quit): ")
    if msg.lower() == 'exit':
        print("Exiting... 👋")
        break
    clean_msg = clean_text(msg)
    vec = vectorizer.transform([clean_msg])
    prob = model.predict_proba(vec)[0][1]  # spam probability
    print(f"Spam Probability: {prob*100:.2f}%")
    print("Result:", "🚨 Spam 🚨" if prob > 0.5 else "✅ Not Spam (Ham)")
    print("-" * 40)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Model Training Complete!

Accuracy: 98.39 %

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.95      0.93      0.94       149

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115


--- SMS Spam Checker ---
Enter an SMS (or 'exit' to quit): click here to get 10000rs
Spam Probability: 25.94%
Result: ✅ Not Spam (Ham)
----------------------------------------
Enter an SMS (or 'exit' to quit): you won a prize money , click here
Spam Probability: 81.85%
Result: 🚨 Spam 🚨
----------------------------------------
Enter an SMS (or 'exit' to quit): click here, get 10000rx
Spam Probability: 30.43%
Result: ✅ Not Spam (Ham)
----------------------------------------
Enter an SMS (or 'exit' to quit): exit
Exiting... 👋
